# Lecture 1:  Data transformation

<div style="border: 1px double black; padding: 10px; margin: 10px">

**Goals for today's lecture:**
* Learn [how to manipulate data](#Data-manipulation), including:
    * [Filtering data](#Row-operation-#1:-Filtering-data)
    * [Arranging (sorting) rows](#Row-operation-#2:-Sorting)
    * [Finding distinct rows](#Row-operation-#3:-distinct())
    * [Selecting subsets of columns](#Column-operation-#1:-select()
    
This lecture note corresponds to [Chapter 4](https://r4ds.hadley.nz/data-transform.html) of your book, and we will also use some ideas from Chapters [14](https://r4ds.hadley.nz/logicals.html)-[15](https://r4ds.hadley.nz/numbers.html).
</div>


## Questions that arose on Piazza
Reminder to please post your questions on Piazza. You will get a faster response, and everyone can see the answers.
- "What's the difference between Github, Colab, and Jupyter?"
- "How is credit assigned for Perusall?"
- "How are the iClicker questions graded?"

## Data manipulation
Manipulating data is an important part of data science--perhaps the most important! As a data scientist, most of your time will be spent simply getting your data into a format that you can analyze:
![data manipulation plot](https://www.datanami.com/wp-content/uploads/2020/07/Anaconda_1.png)
https://www.datanami.com/2020/07/06/data-prep-still-dominates-data-scientists-time-survey-finds/

There are a lot of built-in commands for data manipulation in R:
```{r}
# Traditional data manipulation commands in R
subset()
aggregate()
merge()
reshape()
```
These commands are old and somewhat difficult to use. Instead of the traditional commands, we are going to focus on the `dplyr` package for filtering data. They provide a nice suite of replacements for the traditional commands, which have a consistent, unified interface and interoperate nicely with each other.

The `dplyr` package is part of `tidyverse`, so it is already loaded once we run `library(tidyverse)` (which we remember from last lecture that we should do each time we start R).

We will be using the `nycflights13` data set for this lecture. It does not come with tidyverse. If you are running Jupyter on your own computer you will first need to `install.packages("nycflights13")`.  This data set is about flights departing from the NYC area in 2013.  You will work with part of this data set in Homework 2.

In [2]:
install.packages("nycflights13")


The downloaded binary packages are in
	/var/folders/gh/wbmb7mrx2wb6j56x3zk8lh7m0000gp/T//RtmptAmlg4/downloaded_packages


In [3]:
# install.packages('nycflights13')
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.0 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
print(flights)

# A tibble: 336,776 × 19
    year month   day dep_time sched_de…¹ dep_d…² arr_t…³ sched…⁴ arr_d…⁵ carrier
   <int> <int> <int>    <int>      <int>   <dbl>   <int>   <int>   <dbl> <chr>  
 1  2013     1     1      517        515       2     830     819      11 UA     
 2  2013     1     1      533        529       4     850     830      20 UA     
 3  2013     1     1      542        540       2     923     850      33 AA     
 4  2013     1     1      544        545      -1    1004    1022     -18 B6     
 5  2013     1     1      554        600      -6     812     837     -25 DL     
 6  2013     1     1      554        558      -4     740     728      12 UA     
 7  2013     1     1      555        600      -5     913     854      19 B6     
 8  2013     1     1      557        600      -3     709     723     -14 EV     
 9  2013     1     1      557        600      -3     838     846      -8 B6     
10  2013     1     1      558        600      -2     753     745       8 AA     
# …

Tibble is similar to dataframe and we will learn more about it later in the course.   For now, you can interpret it as a dataframe. 

Notice the types of the variables in `flights`. They include:

* **int** integers
* **dbl** double precision floating point numbers
* **chr** character vectors, or strings
* **dttm** date-time (a date along with a time)

Other types available in R but not represented above include:

* **lgl** logical (either `TRUE` or `FALSE`)
* **fctr** factor (categorical variable with a fixed number of possible values)
* **date** date

## What's a data frame?

Our main goal in R is to work with data, and one of the most fundamental objects in R is the *data frame*. Think of a data frame as a container for a bunch of *vectors* of data:

![dataframe](https://garrettgman.github.io/images/tidy-2.png)

## Row operation #1: Filtering data
The first operation we'll learn about is filtering. Filtering is interpereted to mean "keep only the rows whose columns match these criteria". The syntax for the `filter` command is 
```{r}
filter(<TABLE>, <LOGICAL CRITERIA>)
```
This commands returns a new tibble whose rows all match the specified criteria.

### Types of logical criteria
For those who are new to programming, we now briefly review [wikipedia boolean logic](https://en.wikipedia.org/wiki/Boolean_algebra); [eBook-reference](https://r4ds.hadley.nz/logicals.html#boolean-algebra). The basic logical operators in R are `>`, `>=`, `<`, `<=`, `!=` (not equal), and `==` (equal). The first four are used for comparing numbers and function exactly as in mathematics:
```{r}
> 1 > 1
[1] FALSE
> 1 >= 1
[1] TRUE
> 2.5 < 3
[1] TRUE
> 2.5 <= 3
[1] TRUE
```

### Assignment vs. equality
An extremely common mistake for beginner programmers is to confuse `=` and `==` ("double equals") when writing code. As we have seen,
- `=` is used for
    - assigning a value to a variable, and
    - passing a named parameter into a function. 
- `==` is used for testing equality. 


```{r}
> a = 1  # assigns the integer 1 to a
> b = 2  # assigns the integer 2 to b
> a == 1 # tests that a equals 1
[1] TRUE
> b == 1 # tests that b equals 1
[1] FALSE
```

In [4]:
# Example of assignment vs equality
a = 1
a == 1

[1] TRUE

### Boolean operations
Logical expressions are combined using *boolean operations*. The basic boolean operations are `and`, `or`, and `not`, denoted `&`, `|` and `!` respectively.

In [5]:
! FALSE
FALSE | TRUE
! TRUE

[1] TRUE

[1] TRUE

[1] FALSE

There are also doubled versions of `&` and `|` denoted `&&` and `||`. Do not use these yet. We will return to them later in the course when we discuss programming and control flow.

R abbreviates `TRUE` and `FALSE` as `T` and `F`:

In [6]:
c(T, F)

[1]  TRUE FALSE

### Vectorization
What happens when we ask whether a *vector* satisfies a logical condition? R returns a new logical vector with the same number of entries, showing whether each entry satisfies that condition:

In [7]:
# Examples of logical conditions
a = c(1:10)
print(a)
a > 3

 [1]  1  2  3  4  5  6  7  8  9 10


[1] FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE

### Testing for membership
Another useful operator is `%in%`:
```r
x %in% y
```

return `TRUE` if the value `x` is found in the vector `y`:

In [8]:
"a" %in% c(1, 2, 3)
("a" == 1) | ("a" == 2) | ("a" == 3)

[1] FALSE

[1] FALSE

## Missing data
Something you will often encounter when working with real data are missing observations. R has a special value, `NA` , for representing missing data. You can think of the value of `NA` as "I don't know". Thus, logical and mathematical operations involving `NA` will again return `NA`, so that `NA`s "propagate through" the computation:

In [9]:
NA + 5 # 5 + I-don't-know = I-don't-know

[1] NA

In [10]:
1 < NA  # Is 1 less that I-don't-know? I don't know.

[1] NA

In [11]:
NA < NA  # Is I-don't-know greater than I-don't-know? I don't know.
mean(c(1, NA, 3), na.rm=T) # And so forth.

[1] NA

[1] 2

In [12]:
NA == 1

[1] NA

Since you cannot test `NA`s for equality, R has a special function for determining whether a value is `NA`:

In [13]:
5 == NA

is.na(NA)
is.na(1)

[1] NA

[1] TRUE

[1] FALSE

In [14]:
# Examples of missing values with the ages vector

ages = c(10, 20, 30, NA)
is.na(ages)

[1] FALSE FALSE FALSE  TRUE

### Examples of filtering
Let's use what we have just learned to evaluate some simple queries on the `flights` dataset. Let's first narrow down to all flights that departed on December 31:

In [15]:
# Filter to all flights on 12/31
filter(flights, month==12, day==31)

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,12,31,13,2359,14,439,437,2,B6,839,N566JB,JFK,BQN,189,1576,23,59,2013-12-31 23:00:00
2013,12,31,18,2359,19,449,444,5,DL,412,N713TW,JFK,SJU,192,1598,23,59,2013-12-31 23:00:00
2013,12,31,26,2245,101,129,2353,96,B6,108,N374JB,JFK,PWM,50,273,22,45,2013-12-31 22:00:00
2013,12,31,459,500,-1,655,651,4,US,1895,N557UW,EWR,CLT,95,529,5,0,2013-12-31 05:00:00
2013,12,31,514,515,-1,814,812,2,UA,700,N470UA,EWR,IAH,223,1400,5,15,2013-12-31 05:00:00
2013,12,31,549,551,-2,925,900,25,UA,274,N577UA,EWR,LAX,346,2454,5,51,2013-12-31 05:00:00
2013,12,31,550,600,-10,725,745,-20,AA,301,N3CXAA,LGA,ORD,127,733,6,0,2013-12-31 06:00:00
2013,12,31,552,600,-8,811,826,-15,EV,3825,N14916,EWR,IND,118,645,6,0,2013-12-31 06:00:00
2013,12,31,553,600,-7,741,754,-13,DL,731,N333NB,LGA,DTW,86,502,6,0,2013-12-31 06:00:00


How does all this work? Basically what R does here is create a logical vector that has one entry for each **row** of the input data frame. Then, it returns a new data frame which contains all the rows where the logical vector evaluated to `True`.

## Remark:
Using `==` for testing equality is very important in `R`.  `R` will yield an error if you use `=`.  

In [16]:
# what happens when you forget to use == in filter?
filter(flights, month = 1)

ERROR: ignored

### Counting matches
Sometimes we just want to know how many observations match a given filter. The `nrow()` command can be used to count the number of rows in a data table.

## 🤔 Quiz
How many flights in the table departed in January, February, or March?

<ol style="list-style-type: upper-alpha;">
    <li>Fewer than 50,000</li>
    <li>Between 50,000 and 60,000</li>
    <li>Between 60,000 and 70,000</li>
    <li>Between 70,000 and 80,000</li>
    <li>More than 80,000</li>
</ol>

In [17]:
# Code to get number of flights that departed in Jan-March
nrow(filter(flights, month %in% c(1:3)))

[1] 80789

Let us try to calculate how many flights with missing departure time in our data.

In [18]:
# How many flights have a missing departure time
nrow(filter(flights, is.na(dep_time)))

[1] 8255

How about the number of flights departing between June and July?

In [23]:
# no. of flights departing between June and July
nrow(filter(flights, month %in% c(6, 7)))

[1] 57668

### Row operation #1.5: `head()`/`tail()`
A particularly useful kind of `filter()` is to only keep the first, or last, rows. This happens so often in practical data science, that there are built-in commands to do it:

In [ ]:
# first 6 rows of flights
head(flights)

In [ ]:
# last 6 rows of flights
tail(flights)

## Row operation #2: Sorting
Often we want to sort our data based on one or more column values. This can arise for several reasons:
- The data have some natural order (for example, chronological)
- We want to learn about "extreme" features of the data:
  - "What was the most delayed flight?"
  - "What was the hottest day of the year?"
  - "Who was the tallest NBA player in 2010?"

`arrange()` changes the order of the rows based on the value of the columns. 
- It takes a data frame and a set of column names (or more complicated expressions) to order by. 
- If you provide multiple expressions to order by, it uses the second one to break ties in the first one, third one to break ties in the second one, and so on.
- By default, things are sorted in **ascending** order.

In [ ]:
# sort flights in ascending order by month and day
arrange(flights, month, day)

We sorted the data by month and day, so the top-most rows have the earliest month, folllowed by day.

If you want to sort in **descending** order, write `desc(<column>)` instead:

In [ ]:
# flights in descending order of month
arrange(flights, desc(month))

## 🤔 Quiz
What's the farthest distance traveled by any flight in the dataset?

<ol style="list-style-type: upper-alpha;">
    <li>1200 meters</li>
    <li>2783 miles</li>
    <li>4143 miles</li>
    <li>4983 miles</li>
    <li>4983 km</li>
</ol>

In [ ]:
# solution
arrange(flights, desc(distance))

## Row operation #3: `distinct()`
`distinct()` finds all the unique rows in a dataset. If you supply column names, it will find all unique combinations of those columns.

Here is an example: the number of rows in the dataset is 

In [30]:
nrow(flights)

[1] 336776

In [31]:
nrow(distinct(flights))

[1] 336776

In [4]:
# another example; get all distinct values of origin column
distinct(flights, origin)

origin
<chr>
EWR
LGA
JFK


Obviously there are many flights scheduled for each day. But is there a flight scheduled for *every* day?

In [32]:
# All the unique days in flights
nrow(distinct(flights, day))

[1] 31

## 🤔 Quiz
How many unique *airplanes* are represented in the `flights` table?

<ol style="list-style-type: upper-alpha;">
    <li>3,030</li>
    <li>4,043</li>
    <li>4,044</li>
    <li>90,210</li>
    <li>Can't be determined from the table</li>
</ol>

(Hint: every airplane has a unique tail number.)

In [33]:
# analysis of unique airplanes
nrow(distinct(flights, tailnum))

[1] 4044

## Column operation #1: `select()`
`select()` can help you to narrow down a large dataset by just focusing on the variables you’re interested in. `select()` allows you to rapidly zoom in on a useful subset using operations based on the names of the variables., but you can still get the general idea of how it works:

In [ ]:
tbl <- select(flights, year, month, day, departure_time = dep_time, arrival_time = arr_time)
head(tbl)

In [ ]:
select(flights, tailnum, month)

Note that `select` drops any variables not explicitly mentioned. To just rename some variables while keeping all others, use `rename`.

In [ ]:
head(rename(flights, departure_time = dep_time, arrival_time = arr_time))

If there are a lot of variables, you can save yourself some typing by using `:` and `-` in combination with select. The colon operator selects a range of variables:

In [ ]:
head(select(flights, year:day))

The negative sign lets you select everything but certain columns:

In [ ]:
# select everything except the day column
select(flights, -day)

If you want to bring a few variables at the beginning, you can use `everything()` to refer to the remaining variables.

In [ ]:
# bring dep_time, arr_time, day, month, year to front

head(select(flights, dep_time, arr_time, day, month, year, everything()))

In addition, there are some helper functions that only work inside `select()`.

* `starts_with()`, `ends_with()`, `contains()`
* `matches()`
* `num_range()`

You can consult the documentation or type `?select` at the prompt to learn more about these. Here's just one example of their use.

In [ ]:
# select all the variables that contain the word "time"
head(select(flights, contains('time')))